In [ ]:
from sklearn.linear_model import LogisticRegression 

In [ ]:
class PropensityScore:
    def __init__(selef, df):
        selef.df_data = df.copy()

    def fit(self, X_vars, flag): #flag=treatment col_name
        #calculate propensity score 'e' by Logistic Regression
        self.model = LogisticRegression() 
        self.X = pd.get_dummies(self.df_data[X_vars]).values
        self.y = self.df_data[flag].values
        self.model.fit(self.X, self.y)
        self.df_data["e"] = self.model.predict_proba(self.X)[:, np.where(self.model.classes_ == 1)].flatten()

    def estimate(self, flag, outcome): #flag & outcome =treatment & outcome col_name
        #calculate 'IPWE'(Inverse Probability Weighting Estimator)
        w1 = self.df_data[flag] / self.df_data["e"]
        w0 = (1 - self.df_data[flag]) / (1 - self.df_data["e"]) 
        self.E1 = (np.sum(self.df_data[outcome] * w1)) / np.sum(w1) # E(y1)
        self.E0 = (np.sum(self.df_data[outcome] * w0)) / np.sum(w0) # E(y0)

        #calculate other expectation values
        w1 = self.df_data[flag]
        w0 = (1 - self.df_data[flag])
        self.E11 = (np.sum(self.df_data[outcome] * w1)) / np.sum(w1) # E(y1|z=1)
        self.E00 = (np.sum(self.df_data[outcome] * w0)) / np.sum(w0) # E(y0|z=0)
        self.p1 = np.sum(w1) / np.sum(w0 + w1) # p(z=1)
        self.p0 = np.sum(w0) / np.sum(w0 + w1) # p(z=0)

        self.E10 = (self.E1 - self.E11 * self.p1) / self.p0 if self.p0 != 0 else None # E(y1|z=0)
        self.E01 = (self.E0 - self.E00 * self.p0) / self.p1 if self.p1 != 0 else None # E(y0|z=1)

        self.ATE = self.E1 - self.E0   # E(y1 - y0)
        self.ATT = self.E11 - self.E01 # E(y1 - y0|z=1)
        self.ATU = self.E10 - self.E00 # E(y1 - y0|z=0)


In [ ]:
ps = PropensityScore(df)
ps.fit(["age", "gender"], "x_flag")
ps.estimate("x_flag", "x_rate")